## Installing required Packages

In [ ]:
import pickle
import json
!pip install -U sentence-transformers
import torch

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

     |████████████████████████████████| 61kB 2.9MB/s 
     |████████████████████████████████| 675kB 11.6MB/s 
     |████████████████████████████████| 890kB 42.2MB/s 
     |████████████████████████████████| 1.1MB 43.9MB/s 
     |████████████████████████████████| 3.8MB 46.4MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.6.1-cp36-none-any.whl size=74031 sha256=2c0e8ac30577576959e9298e22d47c6f9896b63e372c8253def706b1f0191549
  Stored in directory: /root/.cache/pip/wheels/d7/fa/17/2b081a8cd8b0a86753fb0e9826b3cc19f0207062c0b2da7008
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=681640ee3d135b1766fb160dc18e0b6b3dbfa94fbdab4fad9e9f1087bf3b5930
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


## Loading & Processing pickle file

In [ ]:
# Enter your path of the pickle file

path_to_text_pickle =  "text_submissions.pickle"

text_submissions = pickle.load( open(path_to_text_pickle, "rb" ) )
print("There are ",len(text_submissions)," submissions")

# Appending the text submissions into a list

text_data = []
for i in text_submissions:
  text_data.append(i[1])



4894


In [ ]:
# Saving

embeddings = model.encode(text_data)
torch.save(embeddings, 'BERT_Embeddings.pt')

# Loading

#embeddings = torch.load('BERT_Embeddings.pt')


##Evaluating Recommendations - Jaccard Distance

In [ ]:
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

def evaluate(list_recommendations,test_input):
  # Finding Average Jaccard Distance
  tot_jaccard = 0
  num_recommendations = len(list_recommendations)
  lem_input = lemmatize_sentence(test_input)
  for recom in list_recommendations:
    lem_topic = lemmatize_sentence(recom)
    temp_jaccard = get_jaccard_sim(lem_input,lem_topic)
    tot_jaccard+=temp_jaccard
  avg_jaccard = tot_jaccard/num_recommendations
  return avg_jaccard
    

   
    
  

## BERT Approach

In [ ]:
embeddings_tensor = torch.zeros(len(embeddings),len(embeddings[0]))
k=0
for li in embeddings:
  embeddings_tensor[k] = torch.tensor(li).view(1,len(embeddings[0]))
  k=k+1
  

In [ ]:
import torch.nn as nn
cosine_score_bert=torch.zeros(len(embeddings),len(embeddings))
sort_score = torch.zeros(len(embeddings),10)
index_score = torch.zeros(len(embeddings),10)

for i in range(len(embeddings)):

  input1 = embeddings_tensor[i].view(1,768)
  input2 = embeddings_tensor
  cos = nn.CosineSimilarity(dim=1, eps=1e-6)
  output = cos(input1, input2)
  # output = torch.cat([output[0:i], output[i+1:]])
  cosine_score_bert[i] = output.view((1,len(embeddings)))


  
  



In [ ]:

# print("Topic numbers range from 0 - 4893\n")
def BERT_recommendation(topic_number):


  #topic_number = 15
  sor,ind = torch.sort(cosine_score_bert[topic_number],descending=True)
  sor = sor[:10]
  ind = ind[:10]
  sort_score = sor.view(1,10)[0]
  
  index_score = ind.view(1,10)

  temp_index = index_score[:5][0]
  # print(temp_index)
  print("Input is\n")
  print(text_submissions[topic_number],"\n\n")
  print("Displaying 10 - Recommended topics\n")
  k=1
  list_recom =[]
  for i in range(len(temp_index)):
    if i>0:


      i = int(temp_index[i].item())
      print("Recommendation -",k)
      print("Similarity is :",sort_score[k],"\n")
      print(text_submissions[i],"\n")
      list_recom.append(text_data[i])
      k=k+1
  avg_jac = evaluate(list_recom,text_data[topic_number])

  print("Average Jaccard Distance Between Input and Recommendations (BERT): ",avg_jac)    








In [ ]:
print("Enter a topic number from 0 - 4894\n")
BERT_recommendation(21)

Enter a topic number from 0 - 4894

Input is

['https://forum.gethopscotch.com//t/variables-not-setting-properly/55261', 'i need help with variables not setting to the correct value/being triggered by the wrong action if anyone would like to'] 


Displaying 10 - Recommended topics

Recommendation - 1
Similarity is : tensor(0.8140) 

['https://forum.gethopscotch.com//t/chocoraspberrys-test-topic/42186', 'I’m  testing making topics with this so please ignore this'] 

Recommendation - 2
Similarity is : tensor(0.8048) 

['https://forum.gethopscotch.com//t/when-bumps-doesn-t-seem-to-work-closed/47478', 'I’m making a game, and “when bumps” doesn’t work for some reason.\nHere are screenshots. When the  bumps zombear, zombear should Di.e .It doesn’t work for some reason.\n'] 

Recommendation - 3
Similarity is : tensor(0.8003) 

['https://forum.gethopscotch.com//t/is-this-a-bug-or-is-my-code-wrong/36162', 'Is this a bug, or is there something wrong with the code?The code looks like this:\nThe t

## TF-IDF Approach

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer().fit_transform(text_data)
tfidf

<4894x19145 sparse matrix of type '<class 'numpy.float64'>'
	with 285174 stored elements in Compressed Sparse Row format>

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import linear_kernel
cosine_score_tfidf = np.zeros((len(text_data),len(text_data)))
for i in range(len(text_data)):

  cosine_similarities = linear_kernel(tfidf[i:i+1], tfidf).flatten()
  # print(cosine_similarities.reshape(1,len(text_data)).shape)
  cosine_score_tfidf[i] = cosine_similarities

cosine_score_tfidf = torch.from_numpy(cosine_score_tfidf)  
#cosine_similarities

In [ ]:

# print("Topic numbers range from 0 - 4893\n")
def TFIDF_recommendation(topic_number):


  #topic_number = 15
  sor,ind = torch.sort(cosine_score_tfidf[topic_number],descending=True)
  sor = sor[:10]
  ind = ind[:10]
  sort_score = sor.view(1,10)[0]
  
  index_score = ind.view(1,10)

  temp_index = index_score[:5][0]
  # print(temp_index)
  print("Input is\n")
  print(text_submissions[topic_number],"\n\n")
  print("Displaying 10 - Recommended topics\n")
  k=1
  list_recom = []
  for i in range(len(temp_index)):
    if i>0:


      i = int(temp_index[i].item())
      print("Recommendation -",k)
      print("Similarity is :",sort_score[k],"\n")
      print(text_submissions[i],"\n")
      list_recom.append(text_data[i])
      k=k+1
  avg_jac = evaluate(list_recom,text_data[topic_number])

  print("Average Jaccard Distance Between Input and Recommendations (TF-IDF) : ",avg_jac)








In [ ]:
print("Enter a topic number from 0 - 4894\n")
TFIDF_recommendation(21)

Enter a topic number from 0 - 4894

Input is

['https://forum.gethopscotch.com//t/variables-not-setting-properly/55261', 'i need help with variables not setting to the correct value/being triggered by the wrong action if anyone would like to'] 


Displaying 10 - Recommended topics

Recommendation - 1
Similarity is : tensor(0.2447, dtype=torch.float64) 

['https://forum.gethopscotch.com//t/how-to-do-basic-variables-hwc-topic-remake/34959', 'Once again, this is from November and hasn’t been read. NOTE: I will go through and add to this. I need to update the images and include self.#Hi, I’m @TheMasterOfAirjitzu.\n##I am going to teach you some cool things to do with variables.Variables are numbers you can change in projects using these blocks:Now, I will show you the default values and what they do.###Default Values\nBoth characters and the device have values. Default values are changed by thing in game, not blocks.Those are the device’s default values.####Width\nWidth is the width of the

## Combination of BERT & TF-IDF

In [ ]:

# print("Topic numbers range from 0 - 4893\n")
def Both_recommendation(topic_number,w1,w2):


  #topic_number = 15

  sor,ind = torch.sort(w1*cosine_score_bert[topic_number]+w2*cosine_score_tfidf[topic_number],descending=True)
  sor = sor[:10]
  ind = ind[:10]
  sort_score = sor.view(1,10)[0]
  
  index_score = ind.view(1,10)

  temp_index = index_score[:5][0]
  # print(temp_index)
  print("Input is\n")
  print(text_submissions[topic_number],"\n\n")
  print("Displaying 10 - Recommended topics\n")
  k=1
  list_recom = []
  for i in range(len(temp_index)):
    if i>0:

      
      i = int(temp_index[i].item())
      print("Recommendation -",k)
      print("Similarity is :",sort_score[k],"\n")
      print(text_submissions[i],"\n")
      list_recom.append(text_data[i])
      k=k+1
  avg_jac = evaluate(list_recom,text_data[topic_number])
  print("Average Jaccard Distance Between Input and Recommendations (Both): ",avg_jac)
      











In [ ]:
print("Enter a topic number from 0 - 4894\n")
w_bert = 0.5
w_tfidf = 0.5 
Both_recommendation(21,w_bert,w_tfidf)

Enter a topic number from 0 - 4894

Input is

['https://forum.gethopscotch.com//t/variables-not-setting-properly/55261', 'i need help with variables not setting to the correct value/being triggered by the wrong action if anyone would like to'] 


Displaying 10 - Recommended topics

Recommendation - 1
Similarity is : tensor(0.4883, dtype=torch.float64) 

['https://forum.gethopscotch.com//t/is-this-a-bug-or-is-my-code-wrong/36162', 'Is this a bug, or is there something wrong with the code?The code looks like this:\nThe two squares should be lined up but they are notIf there’s something wrong with the code then tell me what to do to make it work. If not then…I guess it’s a bug??'] 

Recommendation - 2
Similarity is : tensor(0.4231, dtype=torch.float64) 

['https://forum.gethopscotch.com//t/custom-rules-sometimes-change-behaviour-of-when-blocks/46696', '(NOTE: I don’t know that much about this glitch yet, but I will be editing (or maybe replying to) this topic if I find more information.)I